In [1]:
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
import pathlib
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import glob
from keras.preprocessing import image
import numpy as np
from keras.models import load_model

In [4]:
base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=True, weights="imagenet", input_tensor=None, pooling='max', classes=1000,)
base_output = base_model.output
final_output = Dense(2, activation='softmax')(base_output)

for layer in base_model.layers:
    layer.trainable = False


In [5]:
def make_generator(dataset):
    dataset_path = pathlib.Path('data', dataset)

    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
                                 
    return datagen.flow_from_directory(str(dataset_path),
                                       target_size=(224, 224),
                                       batch_size=32,
                                       shuffle=True,
                                       class_mode='categorical')

In [7]:
train_generator = make_generator('train')
validation_generator = make_generator('test')

model = Model(inputs=base_model.input, outputs=final_output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_generator,
    epochs=5,
    steps_per_epoch=300,
    validation_data=validation_generator,
    validation_steps=16)

model.save('iscute.h5')

Found 10143 images belonging to 2 classes.
Found 1509 images belonging to 2 classes.
Epoch 1/5
300/300 [==============================] - 687s 2s/step - loss: 0.6897 - accuracy: 0.5855 - val_loss: 0.6855 - val_accuracy: 0.6230
Epoch 2/5
300/300 [==============================] - 688s 2s/step - loss: 0.6841 - accuracy: 0.6093 - val_loss: 0.6818 - val_accuracy: 0.5898
Epoch 3/5
300/300 [==============================] - 671s 2s/step - loss: 0.6790 - accuracy: 0.6203 - val_loss: 0.6755 - val_accuracy: 0.6289
Epoch 4/5
300/300 [==============================] - 675s 2s/step - loss: 0.6761 - accuracy: 0.6144 - val_loss: 0.6750 - val_accuracy: 0.5918
Epoch 5/5
300/300 [==============================] - 667s 2s/step - loss: 0.6731 - accuracy: 0.6105 - val_loss: 0.6731 - val_accuracy: 0.6113


In [8]:
model.fit(train_generator,
    epochs=10,
    steps_per_epoch=300,
    validation_data=validation_generator,
    validation_steps=16)

model.save('iscute2.h5')

Epoch 1/10
300/300 [==============================] - 660s 2s/step - loss: 0.6704 - accuracy: 0.6163 - val_loss: 0.6703 - val_accuracy: 0.6074
Epoch 2/10
300/300 [==============================] - 676s 2s/step - loss: 0.6695 - accuracy: 0.6111 - val_loss: 0.6778 - val_accuracy: 0.5781
Epoch 3/10
300/300 [==============================] - 676s 2s/step - loss: 0.6681 - accuracy: 0.6062 - val_loss: 0.6570 - val_accuracy: 0.6562
Epoch 4/10
300/300 [==============================] - 679s 2s/step - loss: 0.6657 - accuracy: 0.6123 - val_loss: 0.6776 - val_accuracy: 0.5566
Epoch 5/10
300/300 [==============================] - 776s 3s/step - loss: 0.6631 - accuracy: 0.6165 - val_loss: 0.6632 - val_accuracy: 0.6152
Epoch 6/10
300/300 [==============================] - 944s 3s/step - loss: 0.6632 - accuracy: 0.6137 - val_loss: 0.6614 - val_accuracy: 0.6035
Epoch 7/10
300/300 [==============================] - 935s 3s/step - loss: 0.6620 - accuracy: 0.6112 - val_loss: 0.6469 - val_accuracy: 0.6602

In [9]:
def predict(model_loaded, filename):
    img = image.load_img(filename, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    return model_loaded.predict (images)

In [20]:
model_loaded = load_model('iscute2.h5')
model_loaded.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

correct = 0
falsePositives = 0
falseNegatives = 0
total = 0
print('not cute, cute')
for filename in glob.glob('data/predict/*'):
    classes = predict(model_loaded, filename)
    if (len(filename) == 33):
        total += 1
        if (classes[0][0] > classes[0][1]):
            correct += 1
            print('correct')
        else:
            falsePositives += 1
    elif(len(filename) == 24):
        total += 1
        if (classes[0][0] < classes[0][1]):
            correct += 1
            print('correct')
        else:
            falseNegatives += 1
    print(filename + ': ' + str(classes))

print('not cute, cute')
print('correct: ' + str(correct) + ' (' + str(correct / total) + '%)')
print('FP: ' + str(falsePositives) + ', FN: ' + str(falseNegatives))

not cute, cute
data/predict\000ada55d36b4bcb.jpg: [[0.1550897  0.84491026]]
data/predict\00a300e8b0cef4d3.jpg: [[0.16990297 0.8300971 ]]
correct
data/predict\00a3654c1cf00d11.jpg: [[0.6231487  0.37685138]]
data/predict\00a36f96e31731c4.jpg: [[0.16352025 0.8364798 ]]
data/predict\00a42a80e5e8d194.jpg: [[0.207901 0.792099]]
data/predict\00a7655d4eabf186.jpg: [[0.19447638 0.80552363]]
data/predict\00abfe9035972732.jpg: [[0.23517293 0.7648271 ]]
data/predict\00ac70478d4727bf.jpg: [[0.11416932 0.88583064]]
correct
data/predict\00acf53b127218c2.jpg: [[0.6291745 0.3708256]]
correct
data/predict\01IrQfV.jpg: [[0.3009802 0.6990198]]
data/predict\01KyAvz.jpg: [[0.67832774 0.3216722 ]]
correct
data/predict\01ZSRgh.jpg: [[0.24337521 0.75662476]]
correct
data/predict\0thB4ca.jpg: [[0.20880605 0.7911939 ]]
correct
data/predict\0TizTVN.jpg: [[0.30732447 0.69267553]]
correct
data/predict\0tOXueK.jpg: [[0.41438872 0.5856112 ]]
correct
data/predict\0TskMIN.jpg: [[0.23276414 0.7672359 ]]
correct
data/pre